# Project 2 - Predicting Energy Output of London Homes. 

Website: https://www.wunderground.com/history/monthly/gb/london/EGLC/date/

### Part 1 - Web Scraping
1. Web Scraping Weather Underground for each month in the years 2012-2013. 
1. Building the DataFrame. 
1. Combining the Dataframes and creating a pickle file



In [1]:
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd
import time
from functools import reduce
import formulas #My own file of separate formulas

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
pd.set_option('display.max_rows', 800)


### Scrape the Website and save it in Soup Files
* Note: I only chose to pull 6 websites at a time to prevent any issues with being locked out

In [3]:
url = 'https://www.wunderground.com/history/monthly/gb/london/EGLC/date/'
soup_2012_01 = formulas.scrape_main(url, '2012-1')
soup_2012_02 = formulas.scrape_main(url, '2012-2')
soup_2012_03 = formulas.scrape_main(url, '2012-3')
soup_2012_04 = formulas.scrape_main(url, '2012-4')
soup_2012_05 = formulas.scrape_main(url, '2012-5')
soup_2012_06 = formulas.scrape_main(url, '2012-6')
# soup_2012_07 = formulas.scrape_main(url, '2012-7')
# soup_2012_08 = formulas.scrape_main(url, '2012-8')
# soup_2012_09 = formulas.scrape_main(url, '2012-9')
# soup_2012_10 = formulas.scrape_main(url, '2012-10')
# soup_2012_11 = formulas.scrape_main(url, '2012-11')
# soup_2012_12 = formulas.scrape_main(url, '2012-12')


In [18]:
soup_2013_01 = formulas.scrape_main(url, '2013-1')
soup_2013_02 = formulas.scrape_main(url, '2013-2')
soup_2013_03 = formulas.scrape_main(url, '2013-3')
soup_2013_04 = formulas.scrape_main(url, '2013-4')
soup_2013_05 = formulas.scrape_main(url, '2013-5')
soup_2013_06 = formulas.scrape_main(url, '2013-6')
# soup_2013_07 = formulas.scrape_main(url, '2013-7')
# soup_2013_08 = formulas.scrape_main(url, '2013-8')
# soup_2013_09 = formulas.scrape_main(url, '2013-9')
# soup_2013_10 = formulas.scrape_main(url, '2013-10')
# soup_2013_11 = formulas.scrape_main(url, '2013-11')
# soup_2013_12 = formulas.scrape_main(url, '2013-12')

### Building the Dataframe 
* As evident below, this ended up being a pretty manual process as many of the 'td' tags did not have any distinguishing class based on columns.  This forced me to use a brute-force approach and pull the index of the entire generated table list to establish the associated columns.  

In [5]:
def day_count(data):
    """Objective: 
    Obtain the amount of rows in the scraped set, which should give you the number of days. 
    """
    day_counter = 0
    for i in data[1:32]:
        item = i.find('td').text.strip()
        if item.isnumeric():
            day_counter+= 1
        else:
            break
    return day_counter

def create_df(data, date):
    """Objective: Build the Dataframe from the scraped data
    Input: Souped object and the Date in YYYY/MM format
    Output: Cleaned Dataframe with all the relavent columns
    """
    dc = day_count(data)
    result = []
    for i in data:
        for j in i.find_all('td'):
            result.append(j.text.strip('  '))
    month = data[0].find('td').text.strip('  ')

    if dc == 31:
        Day = pd.DataFrame(result[1:32], columns=[month])
        Temp_avg = pd.DataFrame(result[36:128:3], columns = ['Temp_Avg'])
        Dew_point = pd.DataFrame(result[132:224:3], columns = ['Dewpoint_avg'])
        Humidity = pd.DataFrame(result[228:320:3], columns = ['Humidity'])
        wind_speed = pd.DataFrame(result[324:416:3], columns = ['Wind_Speed'])
        Pressure = pd.DataFrame(result[420:512:3], columns = ['Pressure'])
        Rain = pd.DataFrame(result[513:], columns = ['Rain'])
    elif dc == 30:
        Day = pd.DataFrame(result[1:31], columns=[month])
        Temp_avg = pd.DataFrame(result[35:124:3], columns = ['Temp_Avg'])
        Dew_point = pd.DataFrame(result[128:217:3], columns = ['Dewpoint_avg'])
        Humidity = pd.DataFrame(result[221:310:3], columns = ['Humidity'])
        wind_speed = pd.DataFrame(result[314:403:3], columns = ['Wind_Speed'])
        Pressure = pd.DataFrame(result[407:496:3], columns = ['Pressure'])
        Rain = pd.DataFrame(result[497:], columns = ['Rain'])
    elif dc == 29:
        Day = pd.DataFrame(result[1:30], columns=[month])
        Temp_avg = pd.DataFrame(result[34:120:3], columns = ['Temp_Avg'])
        Dew_point = pd.DataFrame(result[124:210:3], columns = ['Dewpoint_avg'])
        Humidity = pd.DataFrame(result[214:300:3], columns = ['Humidity'])
        wind_speed = pd.DataFrame(result[304:390:3], columns = ['Wind_Speed'])
        Pressure = pd.DataFrame(result[394:480:3], columns = ['Pressure'])
        Rain = pd.DataFrame(result[481:], columns = ['Rain'])

    elif dc == 28:
        Day = pd.DataFrame(result[1:29], columns=[month])
        Temp_avg = pd.DataFrame(result[33:116:3], columns = ['Temp_Avg'])
        Dew_point = pd.DataFrame(result[120:203:3], columns = ['Dewpoint_avg'])
        Humidity = pd.DataFrame(result[207:290:3], columns = ['Humidity'])
        wind_speed = pd.DataFrame(result[294:377:3], columns = ['Wind_Speed'])
        Pressure = pd.DataFrame(result[381:464:3], columns = ['Pressure'])
        Rain = pd.DataFrame(result[465:], columns = ['Rain'])

    dfs = [Day, Temp_avg, Dew_point, Humidity, wind_speed, Pressure, Rain]
    df_final = reduce(lambda  left,right: pd.merge(left,right,left_index=True,right_index=True), dfs)
    df_final['Date'] = date + '-' + df_final[month]
    df_final.drop([month], axis = 1, inplace = True)
    return df_final


In [4]:
a = formulas.create_df(soup_2012_01, '2012-1')
b = formulas.create_df(soup_2012_02, '2012-2')
c = formulas.create_df(soup_2012_03, '2012-3')
d = formulas.create_df(soup_2012_04, '2012-4')
e = formulas.create_df(soup_2012_05, '2012-5')
f = formulas.create_df(soup_2012_06, '2012-6')
g = formulas.create_df(soup_2012_07, '2012-7')
h = formulas.create_df(soup_2012_08, '2012-8')
i = formulas.create_df(soup_2012_09, '2012-9')
j = formulas.create_df(soup_2012_10, '2012-10') 
k = formulas.create_df(soup_2012_11, '2012-11')
l = formulas.create_df(soup_2012_12, '2012-12')


In [23]:
weather_2012 = pd.concat([a, b, c, d, e, f, g, h, i, j, k, l], ignore_index=True)

In [7]:
weather_2012.head(50)

In [20]:
n = formulas.create_df(soup_2013_01, '2013-1')
o = formulas.create_df(soup_2013_02, '2013-2')
p = formulas.create_df(soup_2013_03, '2013-3')
q = formulas.create_df(soup_2013_04, '2013-4')
r = formulas.create_df(soup_2013_05, '2013-5')
s = formulas.create_df(soup_2013_06, '2013-6')
t = formulas.create_df(soup_2013_07, '2013-7')
u = formulas.create_df(soup_2013_08, '2013-8')
v = formulas.create_df(soup_2013_09, '2013-9')
w = formulas.create_df(soup_2013_10, '2013-10')
x = formulas.create_df(soup_2013_11, '2013-11')
y = formulas.create_df(soup_2013_12, '2013-12')


### Combine the Dataframes

In [25]:
weather_2013 = pd.concat([n, o, p, q, r, s, t, u, v, w, x, y], ignore_index=True)

In [26]:
weather_2013.head(50)

,Temp_Avg,Dewpoint_avg,Humidity,Wind_Speed,Pressure,Rain,Date
0,43.9,37.1,76.5,12.0,29.7,0.00,2013-1-1
1,44.9,41.6,88.2,9.4,30.2,0.00,2013-1-2
2,51.1,48.0,90.7,11.9,30.5,0.00,2013-1-3
3,49.4,44.5,82.6,11.3,30.6,0.00,2013-1-4
4,48.2,45.6,89.8,8.2,30.5,0.00,2013-1-5
5,45.9,44.6,93.7,5.8,30.5,0.00,2013-1-6
6,46.4,43.8,88.8,6.5,30.3,0.00,2013-1-7
7,49.9,45.1,83.3,8.1,30.2,0.00,2013-1-8
8,44.6,41.4,88.3,4.1,30.1,0.00,2013-1-9
9,36.1,34.1,94.3,3.9,30.0,0.00,2013-1-10


In [8]:
weather_2012.plot('Temp_Avg')

### Output the dataframes to Pickle files

In [27]:
weather_2012.to_pickle('Weather_2012.pkl')

In [28]:
weather_2013.to_pickle('Weather_2013.pkl')